In [12]:
import copy

from Game import * 
import params

In [13]:
class Node:
    def __init__(self):
        self.edges = []
        

In [14]:
class Edge:
    def __init__(self, x, y, node = None, p = 0.0):
        self.N = 0
        self.W = 0.0
        self.Q = 0.0
        self.P = p
        self.act_x = x
        self.act_y = y
        
        self.result_node = node       
        

In [15]:
class MCTS:
    def __init__(self, game, evaluator, root_node = None):
        self.root_node = root_node
        if self.root_node == None:
            self.root_node = Node()
        self.game = game
        self.evaluator = evaluator
            
            
    def select_move(self):
        for i in range(params.MTCS_NR):
            g = copy.deep_copy(game)
            self.reach_leaf_node_and_update_edges_and_extend(g)
            
    
        if MCTS_COMP:
            maxN = -1
            arr = []
            for edge in root_node.edges:
                if edge.N > maxN:
                    maxN = edge.N
                    arr=[]
                    arr.append(edge)
                elif edge.N == maxN:
                    arr.append(edge)                    
            edge = np.random.choice(arr)            
        else:
            prob=np.array( len(root_node.edges) )
            for i in range(len(root_node.edges)):
                prob[i] = float(root_node.edges[i].N)**(1.0/MCTS_TAU)
            index = np.random.choice( len(root_node.edges), prob)
            edge = root_node.edges[index]

        g = copy.deep_copy(game)
        g.move(edge.act_x, edge.act_y)
        return edge.act_x, edge.act_y, edge.result_node, g

    def reach_leaf_node_and_update_edges_and_extend(g):
        curr = root_node
        route = []
        while len(curr.edges) != 0:
            prob=np.array( len(curr.edges) )
            for i in range(len(curr.edges)):
                Q = curr.edges[i].Q
                U = MTCS_U_COEFF * curr.edges[i].P / (1.0 + curr.edges[i].N)
                prob[i] = Q + U
            index = np.random.choice( len(curr.edges), prob)
            edge = curr.edges[index]
            route.append(edge)
            curr = edge.result_node
            g.move(edge.act_x, edge.act_y)
        
        p = None
        if g.winner == 1:   # X
            v = 0.0
        elif g.winner == 2: # O
            v = 1.0
        elif g.winner == 3: # Draw
            v = 0.5
        else: # Not terminating state
            p,v = self.evaluator(g)
            
        # Extending the current node
        if p != None:
            for x in range(15):
                for y in  range(15):
                    if g.grid[x,y] == 0:
                        n = Node()
                        e = Edge(x,y,n,p[x,y])
                        curr.edges.append(e)
        
        # Update edges on the route
        for i in range( len(route), -1, -1):
            route[i].N += 1
            route[i].W += v
            route[i].Q = route[i].W / route[i].N
            v = 1 - v 

        return g, curr    
            